In [ ]:
#simulated anealing

In [ ]:
#Heating and cooling for metal

In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import time
import copy

In [38]:
df = pd.read_excel('5geneNetwork.xlsx')
ge_original = np.array(df)
n = len(ge_original[1])
n_of_paramters = 2*n*(n+1)
t = len(ge_original)
min_limit = np.array([[0,0],
                   [0,0],
                   [0,0],
                   [0,0],
                   [0,0]])
max_limit = np.array([[20,20],
                   [20,20],
                   [20,20],
                   [20,20],
                   [20,20]])

In [4]:
#Linear Spline interpolation

step_size = 0.001
H = 0.01
steps = [list(ge_original[0, 0:n])]
for i in range(t-1):
    previous_step = ge_original[i, 0:n]
    next_step = ge_original[i+1, 0:n]
    for j in range(1,int(H/step_size)):
        new_step = list(previous_step + ((next_step-previous_step)/H)*(step_size*j))
        steps.append(new_step)
    steps.append(list(next_step))



spline_check = []
splined_array = steps
for x in range(len(splined_array)):
        if(x%10==0):
            spline_check.append(splined_array[x])
spline_check = np.array(spline_check)

print(spline_check==ge_original)
splined_array= np.array(splined_array)

[[ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]]


In [5]:

Answer = np.array([[5,10,0. ,  0. ,  1. ,  0. , -1, 2. ,  0. ,  0. ,  0. ,  0.],
[10,10,2. ,  0. ,  0. ,  0. ,  0., 0. ,  2. ,  0. ,  0. ,  0.],
[10,10,0. , -1,  0. ,  0. ,  0., 0. , -1,  2. ,  0. ,  0.],
[8,10,0. ,  0 ,  2 ,  0. , -1.,0. ,  0. ,  0. ,  2. ,  0.],
[10,10, 0. ,  0. ,  0. ,  2. ,  0.,0. ,  0. ,  0. ,  0. ,  2.]])
pd.DataFrame(Answer)

,0,1,2,3,4,5,6,7,8,9,10,11
0,5.0,10.0,0.0,0.0,1.0,0.0,-1.0,2.0,0.0,0.0,0.0,0.0
1,10.0,10.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,10.0,10.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,2.0,0.0,0.0
3,8.0,10.0,0.0,0.0,2.0,0.0,-1.0,0.0,0.0,0.0,2.0,0.0
4,10.0,10.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0


In [6]:
def calc_dydt(alpha, beta, g, h, exp ):
    
    
    Generation = alpha * (np.prod(pow(exp,g)))
    Degeneration = beta* (np.prod(pow(exp,h)))
    
    if(np.isnan(Generation) or np.isnan(Degeneration)):
        return(0)
    
    return (Generation-Degeneration)
    
def calc_ge(p):
    
    
    ge_calc = [ge_original[0]]
    H=0.001 #step size
    
    
    for i in range(101):
     
        
        single_exp=[]
       
        for j in range(n):
            
            
            
            cur_p = p[j]
            
            Alpha = cur_p[0]
            Beta = cur_p[1]
            g = cur_p[2:n+2]
            h = cur_p[n+2:(2*(n+1))]
            
            
            
            ge_temp = splined_array[i].copy()
            
            #ge_temp = ge_calc[i].copy()
            value = ge_calc[i][j].copy()
            ge_temp[j]=value
            
            
            k1 = calc_dydt(Alpha,Beta,g,h,ge_temp)
            
            if(k1==0):
                return(0)
            
            k2 = calc_dydt(Alpha,Beta,g,h,ge_temp+(H*0.5*k1))
            
            if(k2==0):
                return(0)
            
            k3 = calc_dydt(Alpha,Beta,g,h,ge_temp+(H*0.5*k2))
            
            if(k3==0):
                return(0)
            
            
            k4 = calc_dydt(Alpha,Beta,g,h,ge_temp+(k3*H))
            
            if(k4==0):
                return(0)
            
            next_step = value + ((1/6)*(k1+2*k2+2*k3+k4)*H)
            
            #next_step = value + (k1*H)

            #rangeKutta Method call
            
            
            
            
            
            single_exp.append(next_step)
            
        ge_calc.append(single_exp)
        
    ge=[]
        
    for i in range (len(ge_calc)):
        if(i%10==0):
            ge.append(list(ge_calc[i]))
    
    ge_calc = np.array(ge)
    
    
    return(ge_calc)


In [7]:
I=5
def fitness(Calculated,cur_p):
    Original = ge_original.copy()
    fit = ((Calculated-Original)/Original)**2
    Limit = (2*n)-I
    #print(Limit)
    #fit = np.sum(abs(Calculated-Original))/np.size(ge_original)
    

    penality = np.sum(np.sort(abs(cur_p[0:n,2:2*(n+1)]),axis=1)[0:n,0:5],axis=1)
    #print(penality)
    c = 0.1
    fit = np.sum(fit, axis=0)
    fit = fit+(penality*c)
    return(fit) 
pd.DataFrame(fitness(calc_ge(Answer),Answer))

,0
0,0.000013
1,0.001613
2,0.000011
3,0.000075
4,0.005512


In [8]:
def mutation(cur,counter):
    
  
    Random_indicies = np.random.randint(0,len(SearchSpace),3)
    
    
    if(counter%20==0.000000000):
        #print("i got here from counter",counter)
        return(trig_mutation(Random_indicies))
    
    else:
        if(phase==1):
            R1=SearchSpace[Random_indicies[0]][0].copy()
        else:
            R1 = cur
        R2 = SearchSpace[Random_indicies[1]][0].copy()
        R3 = SearchSpace[Random_indicies[2]][0].copy()
        
        
        
        return(R1+F*(R2-R3))
    
    
    
    
def trig_mutation(Random_indicies):

    
    R1 = SearchSpace[Random_indicies[0]][0].copy()
    F_R1 = SearchSpace[Random_indicies[0]][1].copy()
    
    R2 = SearchSpace[Random_indicies[1]][0].copy()
    F_R2 = SearchSpace[Random_indicies[1]][1].copy()
        
    R3 = SearchSpace[Random_indicies[2]][0].copy()
    F_R3 = SearchSpace[Random_indicies[2]][1].copy()
           
    
    P = F_R1+F_R2+F_R3
    P1 = F_R1/P
    P2 = F_R2/P
    P3 = F_R3/P
    weighted1,weighted2,weighted3 = R1,R2,R3
    average = (R1+R2+R3)/3
    for i in range(n):
        weighted1[i] = (P2[i]-P1[i])*(R1[i]-R2[i])
        weighted2[i] = (P3[i]-P2[i])*(R2[i]-R3[i])
        weighted3[i] = (P1[i]-P3[i])*(R3[i]-R1[i])
    return(average+weighted1+weighted2+weighted3)
mutation(9,20)

NameError: name 'SearchSpace' is not defined

In [9]:
def crossover(mut,target):
    
    rand_j = np.random.uniform(low=0,high=1,size=target.shape)
    CR_array = np.full(target.shape,CR)
    random_index_i = np.random.randint(0,4)
    random_index_j = np.random.randint(0,11)
    compare = rand_j<=CR_array
    trial = (mut.copy()*compare)+(target.copy()*~compare)
    
    
    
    trial[random_index_i][random_index_j]=mut[random_index_i][random_index_j].copy()
    
    return(trial)   

In [10]:
def check_bounds(trial):
    
    low_bounds = np.concatenate((min_limit, np.full((n,n),-3),np.full((n,n),-3)),axis=1)
    upper_bounds = np.concatenate((max_limit, np.full((n,n),3),np.full((n,n),3)),axis=1)
    
    while( (trial<low_bounds).any() or (trial>upper_bounds).any()):
        trial[trial>upper_bounds]=2*(upper_bounds[trial>upper_bounds])-trial[trial>upper_bounds]
        trial[trial<low_bounds]=2*(low_bounds[trial<low_bounds])-trial[trial<low_bounds]

    
    return(trial)

In [26]:
def random_population():
    random_AB = []
    for i in range(n):
        genewise = []
        for j in range(2):
            genewise.append(np.round(np.random.uniform(low=min_limit[i][j],high =max_limit[i][j]),2))
        random_AB.append(genewise)
        
    AB = np.array(random_AB)        
    gh = np.round(np.random.uniform(low=-3,high=3.1, size=(n,2*n)),2)
    zeros = np.random.randint(low=0,high=2,size=(n,2*n))

    gh = gh*zeros
    Parameters = np.concatenate((AB,gh),axis=1)
    return(Parameters)
def Return_search_space():
    P=100
    SearchSpace = []
    for i in range(P):

        pop = random_population()
        fit = fitness(calc_ge(pop),pop)
        while(type(fit)==type(0.1)):
            pop = random_population()
            fit = fitness(calc_ge(pop),pop)
        SearchSpace.append([pop,fit])
    return(SearchSpace)
#Return_search_space()

In [14]:
def HCLS_old(current):
    trial = current[0:n,2:2*(n+1)].copy()
    selected = current[0:n,2:2*(n+1)].copy()
    AB = current[0:n,0:2].copy()
    sorted_kos = np.sort(abs(selected[selected!=0]))
    if(len(sorted_kos)>25):
        I=25
    else:
        I = len(sorted_kos)
    for i in range(I):
        element = sorted_kos[i]
        
        trial[abs(trial)==element]=0
        
        fitness_trial = fitness(calc_ge(np.concatenate((AB,trial),axis=1)),trial)
        fitness_current = fitness(calc_ge(np.concatenate((AB,selected),axis=1)),selected)
        
        if(any(fitness_trial<fitness_current)):
            #print(element)
            selected = trial.copy()
        else:
            trial = selected.copy()
    #print(current[0:n,2:2*(n+1)][current[0:n,2:2*(n+1)]!=selected])
    return(np.concatenate((AB,selected),axis=1))
def HCLS(current):
    I=5
    trial = current[0:n,2:2*(n+1)].copy()
    selected = current[0:n,2:2*(n+1)].copy()
    AB = current[0:n,0:2].copy()
    trial[trial==0]=4
    sorted_regs = (np.sort(abs(trial)))[0:5,0:5]
    sorted_regs[sorted_regs==4]=0
    
    for j in range(I):
       
        for i in range(n):
          
            current_gene = trial[i].copy()
            current_gene[abs(current_gene)==sorted_regs[i][j]]=0
            trial[i]=current_gene.copy()
        
        fitness_trial = fitness(calc_ge(np.concatenate((AB,trial),axis=1)),trial)
        fitness_current = fitness(calc_ge(np.concatenate((AB,selected),axis=1)),selected)
        
        for k in range(len(fitness_trial)):
            if(fitness_trial[k]<fitness_current[k]):
                selected[k] = trial[k].copy()
            else:
                trial[k] = selected[k].copy()
    return(np.concatenate((AB,selected),axis=1))  

In [15]:
def next_generation(current,count):
    #print("here the counter is :", count)
    mutated = mutation(current,count)
    return(np.round(check_bounds(crossover(mutated,current)),3))
    

In [16]:
def calc_ac(cur):
    calc_gh = np.round(cur[0:n,2:2*(n+1)],0).copy()    
    org_gh=Answer[0:n,2:2*(n+1)].copy()
    TP = np.sum(org_gh[org_gh!=0]==calc_gh[org_gh!=0])
    TN = np.sum(org_gh[org_gh==0]==calc_gh[org_gh==0])
    FP = np.sum(calc_gh[calc_gh!=0]!=org_gh[calc_gh!=0])
    FN = np.sum(calc_gh[org_gh!=0]==0)
    Specificity = TP/(TP+FN)
    Senstivity = TN/(TN+FP)
    Precision = TP/(TP+FP)
    f_score = 2*Precision*Specificity/(Precision+Specificity)
    print(TP,TN,FP,FN)
    return(Precision ,f_score)
    

In [17]:
def refinementAlgorithm(list_of_generations):
    genewise = []
    for j in range(n):
        single_gene = []
        for i in range(len(list_of_generations)):
            single_gene.append(list_of_generations[i][0][j])
        genewise.append(single_gene)
    genewise = np.array(genewise)
    Zeros = np.full(random_population().shape, 1 )
    Zero_count = np.full(random_population().shape, 0)
    
    for i in range(len(genewise)):
        Zero_count[i] =  np.sum(np.logical_and(genewise[i]<=0.25 , genewise[i]>=0) ,axis=0)>=0.9*(len(list_of_generations))
        print(np.sum(np.logical_and(genewise[i]<=0.25 , genewise[i]>=0) ,axis=0))
    Zeros[Zero_count==1]=0
    
    return(Zeros)

In [18]:
def get_10_best(SS):
    temp_SS = SS.copy()
    for i in range(len(temp_SS)):
        temp_SS[i][0]=i
        temp_SS[i][1]=np.sum(temp_SS[i][1])
    temp_SS = np.array(temp_SS)

    ten_percent =(int(0.1*len(SS)))*-1
    print(len(SS))
    return(temp_SS[temp_SS[:,1].argsort(kind='mergesort')][ten_percent:][:,0])  
    
    


In [35]:
def flipOperation(cur_p):
    flipped_Gen = cur_p.copy()
    I=5
    g = flipped_Gen[0:n, 2:n+2].copy()
    h = flipped_Gen[0:n, n+2:(2*(n+1))].copy()
    for j in range(n):
            list_of_spaces=[]
            for i in range(I-1):
                place = np.random.randint(0,n)
                while(place in list_of_spaces):
                    place = np.random.randint(0,n)
                list_of_spaces.append(place)
                tempg = h[j][place]*(-1)
                temph = g[j][place]*(-1)
                g[j][place]=tempg
                h[j][place]=temph
    flipped_Gen[0:n, 2:n+2]=g
    flipped_Gen[0:n, n+2:(2*(n+1))]=h
    
    return(flipped_Gen)

def get_random_one_third(SS):
    thirty_percent = 0.3*(len(SS))
    random_pos = []
    for i in range(int(thirty_percent)):
        pos_= np.random.randint(low=0,high=len(SS))
        while(pos_ in random_pos):
            pos_= np.random.randint(low=0,high=len(SS))
        random_pos.append(pos_)
    #print(set(random_pos))
    return(random_pos)
x = random_population()
print(x)
flipOperation(x)

[[ 9.24  9.39  1.34 -2.6  -2.01  1.17  0.    0.    2.05  0.   -0.79 -0.  ]
 [ 8.18 11.98 -2.16 -0.    0.55 -0.   -1.48 -2.34  2.11  0.   -0.   -0.  ]
 [10.81  6.93 -1.04 -2.31  0.   -2.84  0.    2.   -2.62  2.86 -0.38 -0.  ]
 [ 6.75  7.62  0.   -0.37  0.   -0.39 -1.15 -1.81  0.   -0.   -0.    0.  ]
 [ 6.76  8.8  -0.    0.   -0.    0.    1.32  2.52 -0.   -0.23  2.28 -1.08]]


array([[ 9.24,  9.39, -0.  , -2.05, -2.01,  0.79,  0.  , -1.34,  2.6 ,
         0.  , -1.17, -0.  ],
       [ 8.18, 11.98,  2.34, -0.  , -0.  ,  0.  ,  0.  ,  2.16,  2.11,
        -0.55,  0.  ,  1.48],
       [10.81,  6.93, -2.  ,  2.62, -2.86,  0.38,  0.  ,  1.04,  2.31,
        -0.  ,  2.84, -0.  ],
       [ 6.75,  7.62,  0.  , -0.  ,  0.  ,  0.  , -0.  , -1.81,  0.37,
        -0.  ,  0.39,  1.15],
       [ 6.76,  8.8 , -2.52,  0.  ,  0.23, -2.28,  1.08,  0.  , -0.  ,
         0.  , -0.  , -1.32]])

In [36]:
def next_generation(current,count):
    #print("here the counter is :", count)
    mutated = mutation(current,count)
    result = np.round(check_bounds(crossover(mutated,current)),3)
    result[abs(result)<=0.001]=0
    return(result)
    

In [ ]:
the_best_results =[]
final_population = []

for run in range(5):    
    SearchSpace = Return_search_space()
    G_flip_limit = 0
    best_fitness = np.array([11,11,11,11,11])
    #while(any(best_fitness>0.005)):
    phase=1
    F = 0.5
    CR = 0.8
    all_values=[]
    G_max = 0
    #ll_values = SearchSpace.copy()
    best_values = []
    while(G_max<=850 or any(best_fitness>0.0001)):
        best_values= []
        #print(SearchSpace[0])
        for z in range(len(SearchSpace)):
            #print("Z",z)

            index = z
            count=101

            g = np.array(SearchSpace[index][0]).copy()
            #print(g)
            #rint(g)
            best_value = 0
            best_fitness = float("NaN")
            value = calc_ge(g)
            best_generation = g
            best_fitness = fitness(value,g)
            i=0
            #irst_generation.append([g])

            g = next_generation(best_generation,count)
            Calculated = calc_ge(g)

            while(type(Calculated)==type(0)):
                g = next_generation(g,count)
                Calculated = calc_ge(g)
                if(len(best_values)>3):
                    count=count+1




            new_fitness = fitness(Calculated,g)

            if(any(new_fitness<best_fitness) or type(best_fitness)==type(0.1)):
                #print(z)
                for k in range(len(new_fitness)):

                        if(new_fitness[k]<best_fitness[k]):

                                best_generation[k]=g[k].copy()
                best_value = calc_ge(best_generation)
                best_fitness = fitness(best_value,best_generation)                            





            best_values.append([best_generation,best_fitness])
            #print(len(best_values) )  
            
            






                #rint("for loop",i)
                #rint("best fitness: ",best_fitness)
                #rint(best_generation)

            if(len(best_values)>3):
                count=count+1
            all_values.append([g,new_fitness])
            i=i+1



        print("getting best 10 generations")
        temp = copy.deepcopy(best_values)
        #print(best_values[0])
        Best_10 = get_10_best(temp)

        print(Best_10)
        for pos_ in Best_10:
            pos_ = int(pos_)
            best_values[pos_][0]=HCLS(best_values[pos_][0])
            best_values[pos_][1]=fitness(calc_ge(best_values[pos_][0]),best_values[pos_][0])

        if(G_flip_limit<300):
            if(G_max%300==0):
                G_flip_limit=G_flip_limit+1
                temp = copy.deepcopy(best_values)
                random_30=get_random_one_third(temp)
                print("Generation flipped",random_30,G_flip_limit)
                for pos_ in random_30:
                    pos_ = int(pos_)
                    best_values[pos_][0]=flipOperation(best_values[pos_][0])
                    best_values[pos_][1]=fitness(calc_ge(best_values[pos_][0]),best_values[pos_][0])

        G_max = G_max+1

        SearchSpace = copy.deepcopy(best_values)
        print(run,G_max,z)
        best_g = SearchSpace[0][0].copy()
        best_f = SearchSpace[0][1].copy()
        for i in range(0,len(SearchSpace)):
            f = SearchSpace[i][1].copy()
            gen = SearchSpace[i][0].copy()
            for j in range(len(f)):
                #print(fitness[j],best_fitness[j])
                #print(fitness[j]<best_fitness[j])

                if(f[j]<best_f[j]):
                  #if(f[j]>0.05):
                        best_g[j]=gen[j].copy()
                        best_f[j]=f[j].copy()

        Precision, f_score = calc_ac(best_g)
        print("precision:",Precision)
        print("F-score:",f_score)
        xzy = []
        for i in range(5):
            xzy.append(Answer[i])
            xzy.append(np.round(best_g[i],2))
        print("fitness:",best_f)
        print(pd.DataFrame(xzy))
    the_best_results.append([best_generation,best_fitness])
    final_population.append(SearchSpace)

<ipython-input-6-836ac6d4cbb1>:4: RuntimeWarning: invalid value encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))
<ipython-input-6-836ac6d4cbb1>:5: RuntimeWarning: invalid value encountered in power
  Degeneration = beta* (np.prod(pow(exp,h)))
<ipython-input-6-836ac6d4cbb1>:4: RuntimeWarning: overflow encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))
C:\Users\30391498\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<ipython-input-6-836ac6d4cbb1>:5: RuntimeWarning: overflow encountered in power
  Degeneration = beta* (np.prod(pow(exp,h)))
C:\Users\30391498\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<ipython-input-6-836ac6d4cbb1>:10: RuntimeWarning: invalid value encountered in double_scalars
  return (Generation-Degen

getting best 10 generations
100
[97. 65. 80. 99. 20.  7. 50.  5. 72. 48.]
Generation flipped [73, 27, 41, 39, 53, 19, 3, 62, 71, 81, 9, 89, 6, 64, 86, 80, 15, 63, 65, 2, 14, 20, 37, 92, 87, 97, 69, 78, 23, 56] 1
0 1 99
3 21 19 7
precision: 0.13636363636363635
F-score: 0.18749999999999997
fitness: [0.28330937 0.14302953 0.43333163 0.31065476 0.28538024]
      0      1     2     3     4     5     6     7     8     9     10    11
0   5.00  10.00  0.00  0.00  1.00  0.00 -1.00  2.00  0.00  0.00  0.00  0.00
1  13.72   1.48  0.00 -1.18 -0.74  0.00  2.33  0.00  0.00 -0.19 -1.16  2.98
2  10.00  10.00  2.00  0.00  0.00  0.00  0.00  0.00  2.00  0.00  0.00  0.00
3   7.43   6.70  0.00  0.82  0.00  0.88 -1.66  0.00  2.17  1.66 -0.44 -2.18
4  10.00  10.00  0.00 -1.00  0.00  0.00  0.00  0.00 -1.00  2.00  0.00  0.00
5  17.75  12.44 -1.25  0.00 -0.00  0.07  0.00  0.00  0.00  1.90  0.00  0.00
6   8.00  10.00  0.00  0.00  2.00  0.00 -1.00  0.00  0.00  0.00  2.00  0.00
7  13.80  11.31  0.00  0.00  0.00  0.

<ipython-input-6-836ac6d4cbb1>:4: RuntimeWarning: invalid value encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))
<ipython-input-6-836ac6d4cbb1>:5: RuntimeWarning: invalid value encountered in power
  Degeneration = beta* (np.prod(pow(exp,h)))
<ipython-input-6-836ac6d4cbb1>:4: RuntimeWarning: overflow encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))
C:\Users\30391498\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<ipython-input-6-836ac6d4cbb1>:5: RuntimeWarning: overflow encountered in power
  Degeneration = beta* (np.prod(pow(exp,h)))


getting best 10 generations
100
[54. 69. 80. 24. 92.  6. 37. 20.  2. 78.]
0 2 99
2 16 23 9
precision: 0.08
F-score: 0.1111111111111111
fitness: [0.13050322 0.14302953 0.41462346 0.31065476 0.19684982]
      0      1     2     3     4     5     6     7     8     9     10    11
0   5.00  10.00  0.00  0.00  1.00  0.00 -1.00  2.00  0.00  0.00  0.00  0.00
1   8.36  16.24 -2.20  0.00 -0.20  0.00  0.30  0.00  0.00  0.27  1.65  1.03
2  10.00  10.00  2.00  0.00  0.00  0.00  0.00  0.00  2.00  0.00  0.00  0.00
3   7.43   6.70  0.00  0.82  0.00  0.88 -1.66  0.00  2.17  1.66 -0.44 -2.18
4  10.00  10.00  0.00 -1.00  0.00  0.00  0.00  0.00 -1.00  2.00  0.00  0.00
5   3.88  15.58  2.97 -2.71 -2.39  0.00  2.07  4.00  0.00  0.00  1.62  2.68
6   8.00  10.00  0.00  0.00  2.00  0.00 -1.00  0.00  0.00  0.00  2.00  0.00
7  13.80  11.31  0.00  0.00  0.00  0.00  0.00 -2.25 -2.36  0.00  2.51  2.52
8  10.00  10.00  0.00  0.00  0.00  2.00  0.00  0.00  0.00  0.00  0.00  2.00
9   6.63   1.45 -0.12 -0.21 -1.93  1.86

<ipython-input-6-836ac6d4cbb1>:4: RuntimeWarning: invalid value encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))
<ipython-input-6-836ac6d4cbb1>:5: RuntimeWarning: invalid value encountered in power
  Degeneration = beta* (np.prod(pow(exp,h)))
<ipython-input-6-836ac6d4cbb1>:5: RuntimeWarning: overflow encountered in power
  Degeneration = beta* (np.prod(pow(exp,h)))
C:\Users\30391498\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<ipython-input-6-836ac6d4cbb1>:4: RuntimeWarning: overflow encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))


...

<ipython-input-6-836ac6d4cbb1>:4: RuntimeWarning: invalid value encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))
<ipython-input-6-836ac6d4cbb1>:5: RuntimeWarning: invalid value encountered in power
  Degeneration = beta* (np.prod(pow(exp,h)))


getting best 10 generations
100
[39. 62. 94.  2. 24. 53. 34. 37. 81. 97.]
0 268 99
3 25 15 7
precision: 0.16666666666666666
F-score: 0.21428571428571427
fitness: [6.10835533e-03 4.76848659e-05 5.59605473e-03 5.42365529e-04
 5.25241426e-05]
      0      1     2     3     4     5     6    7     8    9     10    11
0   5.00  10.00  0.00  0.00  1.00  0.00 -1.00  2.0  0.00  0.0  0.00  0.00
1  11.04  13.48  0.19  0.06  0.00  0.00  0.00  0.0  0.36  0.0  2.65  0.31
2  10.00  10.00  2.00  0.00  0.00  0.00  0.00  0.0  2.00  0.0  0.00  0.00
3  10.05  18.18 -0.66 -2.37  0.00  1.89  1.34  0.0  2.65  0.0  0.00  0.00
4  10.00  10.00  0.00 -1.00  0.00  0.00  0.00  0.0 -1.00  2.0  0.00  0.00
5  14.19   8.27 -1.74  0.03 -1.50  0.00  0.28  0.0  0.00  0.0  1.65  0.00
6   8.00  10.00  0.00  0.00  2.00  0.00 -1.00  0.0  0.00  0.0  2.00  0.00
7  14.88  13.83  1.12 -0.72  2.47 -0.25  0.00  0.0  0.00  0.0  2.40  0.00
8  10.00  10.00  0.00  0.00  0.00  2.00  0.00  0.0  0.00  0.0  0.00  2.00
9  12.01   8.98  0.5

<ipython-input-6-836ac6d4cbb1>:4: RuntimeWarning: invalid value encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))
<ipython-input-6-836ac6d4cbb1>:5: RuntimeWarning: invalid value encountered in power
  Degeneration = beta* (np.prod(pow(exp,h)))


getting best 10 generations
100
[74. 39. 62. 94.  2. 24. 53. 34. 37. 81.]
0 269 99
3 25 15 7
precision: 0.16666666666666666
F-score: 0.21428571428571427
fitness: [6.10835533e-03 4.76848659e-05 5.59605473e-03 5.42365529e-04
 5.25241426e-05]
      0      1     2     3     4     5     6    7     8    9     10    11
0   5.00  10.00  0.00  0.00  1.00  0.00 -1.00  2.0  0.00  0.0  0.00  0.00
1  11.04  13.48  0.19  0.06  0.00  0.00  0.00  0.0  0.36  0.0  2.65  0.31
2  10.00  10.00  2.00  0.00  0.00  0.00  0.00  0.0  2.00  0.0  0.00  0.00
3  10.05  18.18 -0.66 -2.37  0.00  1.89  1.34  0.0  2.65  0.0  0.00  0.00
4  10.00  10.00  0.00 -1.00  0.00  0.00  0.00  0.0 -1.00  2.0  0.00  0.00
5  14.19   8.27 -1.74  0.03 -1.50  0.00  0.28  0.0  0.00  0.0  1.65  0.00
6   8.00  10.00  0.00  0.00  2.00  0.00 -1.00  0.0  0.00  0.0  2.00  0.00
7  14.88  13.83  1.12 -0.72  2.47 -0.25  0.00  0.0  0.00  0.0  2.40  0.00
8  10.00  10.00  0.00  0.00  0.00  2.00  0.00  0.0  0.00  0.0  0.00  2.00
9  12.01   8.98  0.5

<ipython-input-6-836ac6d4cbb1>:4: RuntimeWarning: invalid value encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))
<ipython-input-6-836ac6d4cbb1>:5: RuntimeWarning: invalid value encountered in power
  Degeneration = beta* (np.prod(pow(exp,h)))


In [23]:
best_g = SearchSpace[0][0]
best_f = SearchSpace[0][1]
for i in range(0,len(SearchSpace)):
    f = SearchSpace[i][1]
    gen = SearchSpace[i][0]
    for j in range(len(f)):
        #print(fitness[j],best_fitness[j])
        #print(fitness[j]<best_fitness[j])
        
        if(f[j]<best_f[j]):
          #if(f[j]>0.05):
                best_g[j]=gen[j].copy()
                best_f[j]=f[j].copy()
        
Precision, f_score = calc_ac(best_g)
print("precision:",Precision)
print("F-score:",f_score)
xzy = []
for i in range(5):
    xzy.append(Answer[i])
    xzy.append(np.round(best_g[i],3))
print("fitness:",best_f)
pd.DataFrame(xzy)

6 33 8 3
precision: 0.42857142857142855
F-score: 0.5217391304347826
fitness: [1.47866100e-05 1.53571907e-07 5.68977152e-04 3.54328729e-07
 6.81012375e-08]


,0,1,2,3,4,5,6,7,8,9,10,11
0,5.000,10.000,0.000,0.000,1.000,0.000,-1.000,2.0,0.000,0.000,0.000,0.000
1,3.954,8.147,-2.259,1.688,0.000,0.000,-2.590,0.0,-0.279,0.000,0.000,-0.258
2,10.000,10.000,2.000,0.000,0.000,0.000,0.000,0.0,2.000,0.000,0.000,0.000
3,8.870,9.497,1.961,-0.023,0.000,0.027,-0.053,0.0,2.552,0.000,0.000,0.000
4,10.000,10.000,0.000,-1.000,0.000,0.000,0.000,0.0,-1.000,2.000,0.000,0.000
5,7.770,7.619,-2.792,0.000,-0.904,-0.172,0.000,0.0,0.716,-0.759,0.000,0.000
6,8.000,10.000,0.000,0.000,2.000,0.000,-1.000,0.0,0.000,0.000,2.000,0.000
7,7.810,9.727,-0.078,0.000,2.020,0.011,-1.017,0.0,0.000,0.000,2.113,0.000
8,10.000,10.000,0.000,0.000,0.000,2.000,0.000,0.0,0.000,0.000,0.000,2.000
9,10.044,9.606,0.289,-0.211,0.000,1.941,0.120,0.0,0.000,0.000,0.000,1.872


In [530]:
print(SearchSpace)

[[array([[ 6.09 , 11.27 , -2.23 ,  2.96 , -2.12 , -2.2  , -0.   ,  1.3  ,
        -0.   , -0.81 , -0.   ,  3.02 ],
       [ 7.25 ,  8.57 ,  0.   ,  0.   , -0.   ,  0.   , -0.   ,  0.   ,
         1.61 , -0.   , -0.   ,  0.   ],
       [ 5.028, 11.357,  1.34 ,  0.   , -2.408,  0.495, -0.861,  0.   ,
         0.   ,  1.19 ,  0.   ,  0.   ],
       [ 6.016,  8.644,  0.   ,  0.567,  0.   , -1.014,  0.   , -2.176,
         1.68 ,  2.34 , -0.36 ,  1.792],
       [ 9.504,  7.291,  0.   , -0.186,  2.441,  2.677,  0.   ,  0.   ,
         2.95 , -1.98 ,  0.   ,  0.   ]]), array([0.38225924, 0.35845615, 0.01931403, 1.37904952, 0.04511204])], [array([[ 9.49 , 11.   , -0.   , -1.13 ,  1.58 , -1.02 , -0.49 , -1.95 ,
        -1.08 ,  0.   , -2.07 , -0.   ],
       [ 8.157,  9.759,  0.093, -0.576, -0.339,  2.914, -0.447, -0.755,
         2.227,  1.29 ,  0.205,  0.411],
       [ 4.72 ,  9.067,  0.   , -0.753, -1.029,  2.754,  0.087, -0.213,
         0.   ,  0.372, -0.156, -0.405],
       [ 9.233,  8.34

In [422]:
summed_g = the_best_results[0][1].copy()

for i in range(1,len(the_best_results)):

    summed_g = summed_g + the_best_results[i][1]

summed_g = summed_g/len(the_best_results)
Precision, f_score = calc_ac(summed_g)
print("precision:",Precision)
print("F-score:",f_score)
xzy = []
for i in range(5):
    xzy.append(Answer[i])
    xzy.append(np.round(summed_g[i],3))
pd.DataFrame(xzy)

6 28 10 6
precision: 0.375
F-score: 0.42857142857142855


,0,1,2,3,4,5,6,7,8,9,10,11
0,5.000,10.000,0.000,0.000,1.000,0.000,-1.000,2.000,0.000,0.000,0.000,0.000
1,9.571,12.709,0.283,0.000,0.778,-0.290,-0.072,2.263,0.222,0.000,2.507,0.620
2,10.000,10.000,2.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000
3,9.839,11.615,0.343,-0.114,0.000,0.469,-0.007,0.000,2.489,0.000,0.000,0.311
4,10.000,10.000,0.000,-1.000,0.000,0.000,0.000,0.000,-1.000,2.000,0.000,0.000
5,5.274,5.429,-0.286,0.004,-1.254,-0.578,-0.138,0.000,0.160,0.481,0.078,0.123
6,8.000,10.000,0.000,0.000,2.000,0.000,-1.000,0.000,0.000,0.000,2.000,0.000
7,8.774,10.914,0.000,-0.513,2.024,0.000,-0.484,0.000,-2.674,0.000,2.535,2.184
8,10.000,10.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000,0.000,0.000,2.000
9,10.000,10.101,0.000,0.000,-1.351,1.993,0.000,0.000,-0.296,0.000,-0.739,2.181


In [420]:
Refined_generations = []
AB = summed_g[0:n,0:2].copy()
avg_generations = []
Zeros = refinementAlgorithm(best_values)
print(Zeros)

for i in range(len(SearchSpace)):
    SearchSpace[i][0]=Zeros*SearchSpace[i][0]
    SearchSpace[i][1]=fitness(calc_ge(SearchSpace[i][0]),SearchSpace[i][0])
for i in range(len(best_values)):
    best_values[i][0]=Zeros*best_values[i][0]
    best_values[i][1]=fitness(calc_ge(best_values[i][0]),best_values[i][0])

[  0   0  65 107  42  60  82  23  33 109   6  50]
[  0   0  59  24  86   7  99 104   8  99  98  21]
[  0   0  53  89   3  33  74 102  55  96  88  59]
[  0   0 106   1   7 115  33  91   9 102   5   6]
[  0   0 110 107   6   0 110 107  74 100  25   2]
[[1 1 1 0 1 1 1 1 1 0 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 0 1 1 0 1 1 1 1 1 1]
 [1 1 0 0 1 1 0 0 1 1 1 1]]


<ipython-input-357-836ac6d4cbb1>:4: RuntimeWarning: invalid value encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))
<ipython-input-357-836ac6d4cbb1>:5: RuntimeWarning: invalid value encountered in power
  Degeneration = beta* (np.prod(pow(exp,h)))


array([[ 3.876, 15.167, -0.396, -2.343, -0.885,  1.32 ,  2.31 ,  0.467,
        -0.759,  1.03 ,  1.877,  0.135],
       [ 2.893,  3.029, -1.74 , -1.86 ,  0.   ,  0.   ,  0.   ,  0.   ,
         2.747,  0.   ,  0.   ,  0.   ],
       [10.38 , 13.674, -1.094, -1.113,  0.   ,  1.67 ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ],
       [ 5.009,  9.381,  0.726,  2.371, -1.803, -0.117,  0.   ,  2.488,
         0.369, -1.065,  1.02 ,  2.047],
       [ 3.138,  3.504,  0.511,  0.303,  2.393, -1.55 , -0.672,  0.018,
        -0.489,  0.201, -0.567,  0.543]])

In [427]:
G_flip_limit = 0
the_best_results = []
#while(any(best_fitness>0.005)):
phase=1
F = 0.3
CR = 0.5
SearchSpace = best_values.copy()
for y in range(5):
    
    
    index = np.random.randint(0,len(SearchSpace))
    count=101
    all_values = []
    g = SearchSpace[index][0]
    #rint(g)
    best_value = 0
    best_fitness = float("NaN")
    value = calc_ge(g)
    start_time = time.time()
    value = calc_ge(g)
    best_generation = g
    best_fitness = fitness(value,g)
    i=0
    #irst_generation.append([g])
    G_max = 0
    while(G_max<=425):
        g = next_generation(best_generation,count)
        Calculated = calc_ge(g)
        
        while(type(Calculated)==type(0)):
            g = next_generation(g,count)
            Calculated = calc_ge(g)
            if(len(best_values)>3):
                count=count+1
      
        
        new_fitness = fitness(Calculated,g)
        
        if(any(new_fitness<best_fitness) or type(best_fitness)==type(0.1)):
        
            for k in range(len(new_fitness)):
               #if(new_fitness[k]>0.05):
                    if(new_fitness[k]<best_fitness[k]):
                        
                        best_generation[k]=g[k].copy()
           
            
            
            best_value = calc_ge(best_generation)
            best_fitness = fitness(best_value,best_generation)
            best_values.append([best_generation,best_fitness])
            #SearchSpace.append([best_generation,best_fitness])
            
            
        
            #print("for loop",i)
            #print("best fitness: ",best_fitness)
            #print(best_generation)
        
        if(len(best_values)>3):
            count=count+1
        all_values.append([g,new_fitness])
        i=i+1
        G_max = G_max+1
        
    all_best_values.append(best_values)
    best_fitness = fitness(calc_ge(best_generation),best_generation)
    the_best_results.append([best_fitness,best_generation])
    total_time_taken= time.time() - start_time
    print("==========================================================")    
    print("loop has ended")
    print("==========================================================")    
    print("results:")
    print(calc_ac(best_generation))
    print("total time taken: ", total_time_taken/60, "mins")
    print("best fitness: ",best_fitness)
    print(best_generation)

<ipython-input-357-836ac6d4cbb1>:4: RuntimeWarning: invalid value encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))
<ipython-input-357-836ac6d4cbb1>:5: RuntimeWarning: invalid value encountered in power
  Degeneration = beta* (np.prod(pow(exp,h)))


loop has ended
results:
2 27 15 6
(0.11764705882352941, 0.15999999999999998)
total time taken:  0.20836795568466188 mins
best fitness:  [0.00560347 0.00058652 0.00169295 0.01167993 0.00015213]
[[ 6.523 10.284 -2.562  0.     1.809 -0.728  0.     0.     0.     0.
   2.07   1.22 ]
 [ 8.264  9.132  0.187  0.229  0.     0.418 -0.392  0.     2.93   0.
   0.     0.   ]
 [ 7.484  8.282  2.153  0.    -0.741 -1.479  0.265  0.     0.772  0.
   0.     0.   ]
 [ 9.575  7.151  0.2    2.944  2.507 -2.107 -2.604  0.     0.     0.
   0.     0.   ]
 [10.092  8.658  0.28   0.     0.     1.689  0.    -1.356  0.    -0.015
   0.     1.775]]
loop has ended
results:
4 28 13 5
(0.23529411764705882, 0.30769230769230765)
total time taken:  0.20855242411295574 mins
best fitness:  [1.32231480e-02 6.43412065e-04 1.52977201e-03 1.40747716e-02
 7.45421092e-05]
[[ 4.912e+00  9.812e+00 -2.812e+00  0.000e+00  2.440e+00 -1.081e+00
   0.000e+00  0.000e+00  0.000e+00  2.558e+00  0.000e+00  1.445e+00]
 [ 8.297e+00  9.251e+0

<ipython-input-357-836ac6d4cbb1>:4: RuntimeWarning: overflow encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))
C:\Users\30391498\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


loop has ended
results:
2 28 14 6
(0.125, 0.16666666666666666)
total time taken:  0.21302653948465983 mins
best fitness:  [9.45662982e-03 6.37946790e-04 1.49791046e-03 1.19578180e-02
 6.50911146e-05]
[[ 7.2560e+00  1.1969e+01 -2.9980e+00  0.0000e+00  2.9830e+00 -1.2730e+00
   0.0000e+00  0.0000e+00  0.0000e+00  1.6700e-01  0.0000e+00  1.0100e+00]
 [ 8.3010e+00  9.3490e+00  2.0000e-03  1.7600e-01  0.0000e+00  4.9300e-01
  -3.6700e-01  0.0000e+00  2.9300e+00  0.0000e+00  0.0000e+00  0.0000e+00]
 [ 7.3710e+00  8.2820e+00  2.1080e+00  0.0000e+00 -7.4100e-01 -1.4790e+00
   2.6500e-01  0.0000e+00  7.7200e-01  0.0000e+00  0.0000e+00  0.0000e+00]
 [ 9.5750e+00  7.2080e+00  2.0000e-01  2.8370e+00  2.5700e+00 -2.1070e+00
  -2.5220e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00]
 [ 9.9820e+00  1.0720e+01  1.0000e-01  0.0000e+00  0.0000e+00  1.9100e+00
   0.0000e+00 -1.2530e+00  0.0000e+00  0.0000e+00  0.0000e+00  2.2940e+00]]
loop has ended
results:
2 27 15 6
(0.11764705882352941

In [84]:
AB_generations = []
best_values = []
count = 101
phase=2
F = 0.1
CR = 0.5
for x in range(1):
    
    best_generation = best_g.copy()
    best_fitness = fitness(calc_ge(best_generation),best_generation)
    print("starting from fitness: ",best_fitness)
    print(calc_ac(best_generation))
    print(best_generation)
    for i in range(500):
        g = next_generation(best_generation,count)
        Calculated = calc_ge(g)
        
        while(type(Calculated)==type(0)):
            g = next_generation(g,count)
            Calculated = calc_ge(g)
            
            if(len(best_values)>3):
                count=count+1
        
        new_fitness= fitness(calc_ge(g),g)
        if(any(new_fitness<best_fitness)):
        
            for k in range(len(new_fitness)):
                if(new_fitness[k]<best_fitness[k]):
                    if(new_fitness[k]>0.04):
                        best_generation[k]=g[k].copy()
            best_generation =  HCLS(best_generation)
            best_value = calc_ge(best_generation)
            best_fitness = fitness(best_value,best_generation)
            best_values.append([best_fitness,best_generation])
            
            #print("for loop",i)
            #print("best fitness: ",best_fitness)
            #print(best_generation)
            
        if(len(best_values)>3):
            count=count+1
            
    AB_generations.append([best_fitness,best_generation])  
    
    print("==========================================================")    
    print("loop has ended")
    print("==========================================================")    
    print("results:")
    print("best fitness: ",best_fitness)
    print("accuracy Parameters")
    print(calc_ac(best_generation))
    print(best_generation)

starting from fitness:  [0.05986056 0.09009475 0.05282497 0.08579937 0.07860809]
6 29 9 6
(0.4, 0.4444444444444445)
[[ 3.202  6.407 -0.    -0.362 -0.     0.    -0.612  1.785  0.    -0.
   0.41   0.   ]
 [10.116  9.044  0.405  0.     0.604  0.     0.    -2.428  2.639  0.
   0.     0.   ]
 [ 6.301  6.481  0.     0.    -0.709 -0.52   0.     0.     0.     2.4
   0.     0.   ]
 [ 8.144  7.63   2.647  0.     0.971  0.063  0.     0.     0.     0.
   2.109  0.   ]
 [ 9.963  9.105  0.321 -0.372  0.     2.49   0.     0.    -0.795  0.
   0.948  1.729]]


<ipython-input-5-d1e525efa95f>:4: RuntimeWarning: invalid value encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))
<ipython-input-5-d1e525efa95f>:5: RuntimeWarning: invalid value encountered in power
  Degeneration = beta* (np.prod(pow(exp,h)))


loop has ended
results:
best fitness:  [0.04015734 0.06953823 0.03314909 0.06401305 0.0696713 ]
accuracy Parameters
5 32 7 6
(0.4166666666666667, 0.43478260869565216)
[[ 3.376  6.75  -0.157 -0.087  0.018 -0.067 -0.636  1.405  0.159  0.
   0.409  0.081]
 [10.531  9.317  0.622  0.     0.36   0.016  0.    -2.157  2.352  0.
   0.     0.   ]
 [ 6.353  6.368  0.     0.    -1.31  -0.286  0.     0.     0.126  0.
  -0.347  0.   ]
 [ 8.144  7.733  2.366  0.     0.918  0.282 -0.116  0.     0.     0.
   2.214  0.082]
 [ 9.941  9.389  0.318 -0.3    0.     2.494  0.     0.    -0.434  0.
   0.945  1.715]]


6 33 6 5
precision: 0.5
F-score: 0.5217391304347826
fitness: [0.03445712 0.05458189 0.03640068 0.0703794  0.05150141]


<ipython-input-5-d1e525efa95f>:4: RuntimeWarning: invalid value encountered in power
  Generation = alpha * (np.prod(pow(exp,g)))
<ipython-input-5-d1e525efa95f>:5: RuntimeWarning: invalid value encountered in power
  Degeneration = beta* (np.prod(pow(exp,h)))


,0,1,2,3,4,5,6,7,8,9,10,11
0,5.000,10.000,0.000,0.000,1.000,0.000,-1.000,2.000,0.000,0.000,0.000,0.000
1,6.978,11.552,-0.766,0.021,0.004,-0.298,-0.110,0.542,-0.083,0.000,-0.001,0.599
2,10.000,10.000,2.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000
3,14.799,12.820,2.419,-0.160,0.000,0.023,0.404,-0.006,1.977,-0.016,0.003,0.109
4,10.000,10.000,0.000,-1.000,0.000,0.000,0.000,0.000,-1.000,2.000,0.000,0.000
5,10.484,11.410,-0.032,-0.390,-0.556,-0.008,0.014,0.000,-0.114,2.061,-0.044,0.092
6,8.000,10.000,0.000,2.000,0.000,0.000,-1.000,0.000,0.000,0.000,2.000,0.000
7,7.963,8.474,1.701,1.188,0.000,0.000,-0.406,0.226,-0.100,0.148,2.270,0.353
8,10.000,10.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000,0.000,0.000,2.000
9,9.717,9.639,0.011,0.003,0.000,2.419,-0.033,0.045,0.014,0.000,0.038,2.223


In [279]:
End_Result = []

In [ ]:
End_Result.append([best_generation,best_fitness])

In [ ]:

        
Precision, f_score = calc_ac(best_generation)
print("precision:",Precision)
print("F-score:",f_score)
xzy = []
for i in range(5):
    xzy.append(Answer[i])
    xzy.append(np.round(best_generation[i],3))
print("fitness:",best_fitness)
pd.DataFrame(xzy)
